In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install gensim==4.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 52.6 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2


In [ ]:
pip install --upgrade gensim scipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip freeze | grep gensim

gensim==4.2.0


In [3]:
pip install scipy==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4


In [4]:
!pip freeze | grep scipy

scipy==1.10.1


In [5]:
!pip install tensorflow

In [60]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Input, SimpleRNN, LSTM, Dense, Masking, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [61]:
X = pd.read_csv("/home/hailinh/code/szaboildi/uk-pol-speech-classifier/processed_data/features_1000sample_400min_600cutoff.csv")
y = pd.read_csv("/home/hailinh/code/szaboildi/uk-pol-speech-classifier/processed_data/target_1000sample_400min_600cutoff.csv")

In [62]:
X

,text
0,out of parent of eligible twoyearolds have tak...
1,with permission madam speaker i would like to ...
2,do but i know it to some extent because my dut...
3,great debate but it is a debate about economic...
4,should think that i have changed my interest c...
...,...
6995,dokeep inflation lowthere is merit in that in ...
6996,roughly in northern ireland between protestant...
6997,to british act of parliament and the constitut...
6998,make a total commitment to the principle of no...


In [63]:
X.shape

(7000, 1)

In [64]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# Initialize list
tokenized_texts = []

# Tokenize each text per row
for text_row in X['text']:
    tokenized_text = text_to_word_sequence(text_row)
    tokenized_texts.append(tokenized_text)


In [65]:
len(tokenized_texts)

7000

In [66]:
X_tokenized = pd.DataFrame({'tokenized_text': tokenized_texts})
X_tokenized = X_tokenized["tokenized_text"]
X_tokenized.shape

(7000,)

In [67]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

# Split in train and test set
X_train, X_test, y_train, y_test = train_test_split(X_tokenized, y, test_size=0.2, random_state=42)

In [68]:
import gensim.downloader as api
word2vec_model = api.load("glove-wiki-gigaword-50")

In [69]:
import nltk
from nltk.corpus import stopwords

# Download English stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hailinh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word not in stop_words and word in word2vec:
            embedded_sentence.append(word2vec[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

In [71]:
# Embed the training and test sentences
X_train_embed = embedding(word2vec_model, X_train)
X_test_embed = embedding(word2vec_model, X_test)

In [72]:
X_train_embed

[array([[ 0.30577 ,  0.30053 ,  0.22806 , ...,  0.70866 , -0.039504,
          0.18456 ],
        [ 0.53662 , -0.01189 , -0.24985 , ..., -0.23625 ,  0.063342,
          0.17774 ],
        [ 0.13627 , -0.054478,  0.3703  , ...,  0.17399 , -0.19338 ,
          0.51921 ],
        ...,
        [-0.13531 ,  0.15485 , -0.07309 , ..., -0.028168, -0.18904 ,
         -0.6769  ],
        [ 0.13627 , -0.054478,  0.3703  , ...,  0.17399 , -0.19338 ,
          0.51921 ],
        [-0.097021, -0.41827 , -0.034515, ...,  0.88056 ,  0.73765 ,
         -0.53865 ]], dtype=float32),
 array([[-0.71224 , -0.12369 , -0.48733 , ..., -0.027037,  0.59435 ,
         -0.48785 ],
        [ 0.25628 , -0.080969, -0.22797 , ..., -0.63826 , -0.071853,
          0.30162 ],
        [-0.45357 , -0.61985 ,  0.064357, ...,  0.5309  , -0.16925 ,
          0.36956 ],
        ...,
        [ 0.010514, -0.28378 ,  0.10601 , ..., -0.49353 , -0.14949 ,
         -0.73603 ],
        [-0.22258 ,  0.089765,  0.45857 , ..., -0.23081 ,

In [73]:
text_lengths = [len(text) for text in X_train_embed]
max_len = max(text_lengths)
max_len

350

In [74]:
# Pad embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=max_len)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=max_len)

In [75]:
y_train = y_train["party"]
y_test = y_test["party"]

In [76]:
from sklearn.preprocessing import LabelEncoder

# Initialize Label encoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [77]:
# One-hot-encode y_train and y_test

from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train_encoded, num_classes=7)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=7)

In [ ]:
# Initialize model and compile
model = Sequential()
model.add(LSTM(units=128))
model.add(Dense(10, activation = "relu"))
model.add(Dense(7, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define callbacks
# Path for modelCheckpoint
checkpoint_path = 'best_model.keras'
modelCheckpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True)

LRreducer = ReduceLROnPlateau(patience=3, monitor='val_loss', factor=0.1, min_lr=0, verbose=1)

EarlyStopper = EarlyStopping(patience=10, restore_best_weights = True, monitor='val_loss', verbose=1)

In [ ]:
# Fit
history = model.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=30,
    validation_split=0.2,
    callbacks=[EarlyStopper],
    verbose=1
)

Epoch 1/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 135ms/step - accuracy: 0.1387 - loss: 1.9451 - val_accuracy: 0.1464 - val_loss: 1.9459
Epoch 2/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 134ms/step - accuracy: 0.1512 - loss: 1.9459 - val_accuracy: 0.1411 - val_loss: 1.9465
Epoch 3/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 21s 149ms/step - accuracy: 0.1442 - loss: 1.9445 - val_accuracy: 0.1411 - val_loss: 1.9464
Epoch 4/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 23s 163ms/step - accuracy: 0.1488 - loss: 1.9442 - val_accuracy: 0.1473 - val_loss: 1.9469
Epoch 5/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 23s 161ms/step - accuracy: 0.1533 - loss: 1.9444 - val_accuracy: 0.1473 - val_loss: 1.9486
Epoch 6/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 23s 161ms/step - accuracy: 0.1465 - loss: 1.9445 - val_accuracy: 0.1429 - val_loss: 1.9484
Epoch 7/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 157ms/step - accuracy: 0.1524 - loss: 1.9437 - val_accuracy: 0.1446 - val_loss: 1.9509
Epoch 8/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 159ms/step - accuracy: 0.1502 - loss: 1

In [ ]:
# Initialize model_2 and compile
model_2 = Sequential()
model_2.add(LSTM(units=128, activation = "tanh"))
model_2.add(Dense(30, activation = "relu"))
model_2.add(Dense(10, activation = "relu"))
model_2.add(Dense(7, activation="softmax"))

model_2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit model_2
history_2 = model_2.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=50,
    validation_split=0.2,
    callbacks=[EarlyStopper],
    verbose=1
)

Epoch 1/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 132ms/step - accuracy: 0.1504 - loss: 1.9405 - val_accuracy: 0.1473 - val_loss: 1.9487
Epoch 2/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 142ms/step - accuracy: 0.1513 - loss: 1.9441 - val_accuracy: 0.1509 - val_loss: 1.9315
Epoch 3/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 160ms/step - accuracy: 0.1602 - loss: 1.9306 - val_accuracy: 0.1661 - val_loss: 1.9239
Epoch 4/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 23s 166ms/step - accuracy: 0.1624 - loss: 1.9314 - val_accuracy: 0.1625 - val_loss: 1.9259
Epoch 5/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 182ms/step - accuracy: 0.1763 - loss: 1.9251 - val_accuracy: 0.1536 - val_loss: 1.9058
Epoch 6/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 171ms/step - accuracy: 0.1822 - loss: 1.9153 - val_accuracy: 0.1321 - val_loss: 1.9482
Epoch 7/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 173ms/step - accuracy: 0.1405 - loss: 1.9453 - val_accuracy: 0.2018 - val_loss: 1.9265
Epoch 8/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 168ms/step - accuracy: 0.1624 - loss: 1

In [ ]:
history_2.history['accuracy']

[0.15245535969734192,
 0.15468749403953552,
 0.16272321343421936,
 0.16674107313156128,
 0.17812499403953552,
 0.16718749701976776,
 0.14598214626312256,
 0.17343750596046448,
 0.21696428954601288,
 0.24017857015132904]

In [ ]:
EarlyStopper = EarlyStopping(patience=10, restore_best_weights = True, monitor='val_accuracy', verbose=1)

In [ ]:
# Fit model_2 with newly defined EarlyStopper
history_2 = model_2.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=50,
    validation_split=0.2,
    callbacks=[EarlyStopper],
    verbose=1
)

Epoch 1/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 133ms/step - accuracy: 0.1610 - loss: 1.9348 - val_accuracy: 0.1455 - val_loss: 1.9347
Epoch 2/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 137ms/step - accuracy: 0.1632 - loss: 1.9312 - val_accuracy: 0.1732 - val_loss: 1.9287
Epoch 3/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 154ms/step - accuracy: 0.1786 - loss: 1.9307 - val_accuracy: 0.1804 - val_loss: 1.9272
Epoch 4/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 157ms/step - accuracy: 0.1764 - loss: 1.9297 - val_accuracy: 0.1813 - val_loss: 1.9345
Epoch 5/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 160ms/step - accuracy: 0.1815 - loss: 1.9329 - val_accuracy: 0.1813 - val_loss: 1.9306
Epoch 6/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 23s 165ms/step - accuracy: 0.1711 - loss: 1.9298 - val_accuracy: 0.1473 - val_loss: 1.9664
Epoch 7/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 171ms/step - accuracy: 0.1659 - loss: 1.9383 - val_accuracy: 0.1705 - val_loss: 1.9326
Epoch 8/50
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 177ms/step - accuracy: 0.1948 - loss: 1

In [ ]:
# Initialize model_3 and compile
model_3 = Sequential()
model_3.add(LSTM(units=128, activation = "tanh"))
model_3.add(Dense(50, activation = "relu"))
model_3.add(Dense(30, activation = "relu"))
model_3.add(Dense(10, activation = "relu"))
model_3.add(Dense(7, activation="softmax"))

model_3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
EarlyStopper_3 = EarlyStopping(patience=12, restore_best_weights = True, monitor='val_accuracy', verbose=1)

In [ ]:
# Fit model_3
history_3 = model_3.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    callbacks=[EarlyStopper_3],
    verbose=1
)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 139ms/step - accuracy: 0.1609 - loss: 1.9431 - val_accuracy: 0.1562 - val_loss: 1.9442
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 141ms/step - accuracy: 0.1649 - loss: 1.9414 - val_accuracy: 0.1518 - val_loss: 1.9342
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 23s 164ms/step - accuracy: 0.1773 - loss: 1.9362 - val_accuracy: 0.1813 - val_loss: 1.9327
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 177ms/step - accuracy: 0.1641 - loss: 1.9364 - val_accuracy: 0.1580 - val_loss: 1.9379
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 177ms/step - accuracy: 0.1639 - loss: 1.9332 - val_accuracy: 0.1437 - val_loss: 1.9459
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 174ms/step - accuracy: 0.1619 - loss: 1.9376 - val_accuracy: 0.1786 - val_loss: 1.9310
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 179ms/step - accuracy: 0.1715 - loss: 1.9321 - val_accuracy: 0.1795 - val_loss: 1.9312
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 180ms/step - accuracy: 0.1699 -

In [24]:
EarlyStopper_4 = EarlyStopping(patience=15, restore_best_weights = True, monitor='val_accuracy', verbose=1)

In [25]:
# Initialize model_4 and compile
model_4 = Sequential()
model_4.add(LSTM(units=128, activation = "tanh"))
model_4.add(Dense(75, activation = "relu"))
model_4.add(Dense(50, activation = "relu"))
model_4.add(Dense(30, activation = "relu"))
model_4.add(Dense(10, activation = "relu"))
model_4.add(Dense(7, activation="softmax"))

model_4.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Fit model_4
history_4 = model_4.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    callbacks=[EarlyStopper_4],
    verbose=1
)

Epoch 1/100
140/140 [==============================] - 63s 423ms/step - loss: 1.9466 - accuracy: 0.1350 - val_loss: 1.9450 - val_accuracy: 0.1491
Epoch 2/100
140/140 [==============================] - 61s 435ms/step - loss: 1.9444 - accuracy: 0.1449 - val_loss: 1.9463 - val_accuracy: 0.1455
Epoch 3/100
140/140 [==============================] - 58s 418ms/step - loss: 1.9442 - accuracy: 0.1411 - val_loss: 1.9465 - val_accuracy: 0.1464
Epoch 4/100
140/140 [==============================] - 62s 443ms/step - loss: 1.9426 - accuracy: 0.1404 - val_loss: 1.9443 - val_accuracy: 0.1500
Epoch 5/100
140/140 [==============================] - 64s 455ms/step - loss: 1.9407 - accuracy: 0.1464 - val_loss: 1.9437 - val_accuracy: 0.1357
Epoch 6/100
140/140 [==============================] - 61s 439ms/step - loss: 1.9404 - accuracy: 0.1589 - val_loss: 1.9457 - val_accuracy: 0.1429
Epoch 7/100
140/140 [==============================] - 58s 415ms/step - loss: 1.9351 - accuracy: 0.1489 - val_loss: 1.9320 -

In [ ]:
# Initialize model_5 and compile
model_5 = Sequential()
model_5.add(LSTM(units=256, activation = "tanh"))
model_5.add(Dense(75, activation = "relu"))
model_5.add(Dense(50, activation = "relu"))
model_5.add(Dense(30, activation = "relu"))
model_5.add(Dense(10, activation = "relu"))
model_5.add(Dense(7, activation="softmax"))

model_5.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit model_5
history_5 = model_5.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    callbacks=[EarlyStopper_4],
    verbose=1
)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 40s 275ms/step - accuracy: 0.1382 - loss: 1.9461 - val_accuracy: 0.1420 - val_loss: 1.9462
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 293ms/step - accuracy: 0.1375 - loss: 1.9454 - val_accuracy: 0.1732 - val_loss: 1.9428
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 44s 315ms/step - accuracy: 0.1467 - loss: 1.9425 - val_accuracy: 0.1429 - val_loss: 1.9462
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 321ms/step - accuracy: 0.1599 - loss: 1.9445 - val_accuracy: 0.1464 - val_loss: 1.9458
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 323ms/step - accuracy: 0.1508 - loss: 1.9421 - val_accuracy: 0.1527 - val_loss: 1.9363
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 323ms/step - accuracy: 0.1688 - loss: 1.9347 - val_accuracy: 0.1455 - val_loss: 1.9461
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 48s 343ms/step - accuracy: 0.1739 - loss: 1.9318 - val_accuracy: 0.1723 - val_loss: 1.9372
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 52s 372ms/step - accuracy: 0.1960 -

In [78]:
embedding_dim = word2vec_model.vector_size
embedding_dim

50

In [ ]:
model_6 = Sequential()
model_6.add(Input(shape=(max_len, embedding_dim)))
model_6.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model_6.add(Dense(units=7, activation='softmax'))

model_6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit model_6
history_6 = model_6.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    callbacks=[EarlyStopper_4],
    verbose=1
)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 21s 144ms/step - accuracy: 0.1379 - loss: 1.9461 - val_accuracy: 0.1670 - val_loss: 1.9413
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 144ms/step - accuracy: 0.1470 - loss: 1.9423 - val_accuracy: 0.1545 - val_loss: 1.9385
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 156ms/step - accuracy: 0.1515 - loss: 1.9408 - val_accuracy: 0.1661 - val_loss: 1.9394
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 21s 153ms/step - accuracy: 0.1734 - loss: 1.9335 - val_accuracy: 0.1866 - val_loss: 1.9244
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 21s 153ms/step - accuracy: 0.1747 - loss: 1.9288 - val_accuracy: 0.1750 - val_loss: 1.9089
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 157ms/step - accuracy: 0.1744 - loss: 1.9253 - val_accuracy: 0.1848 - val_loss: 1.9167
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 23s 167ms/step - accuracy: 0.1946 - loss: 1.8963 - val_accuracy: 0.2009 - val_loss: 1.8794
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 168ms/step - accuracy: 0.2003 -

In [ ]:
model_7 = Sequential()
model_7.add(Input(shape=(max_len, embedding_dim)))
model_7.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model_7.add(Dense(units=50, activation='relu'))
model_7.add(Dense(units=7, activation='softmax'))

model_7.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit model_7
history_7 = model_7.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    callbacks=[EarlyStopper_4],
    verbose=1
)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 21s 143ms/step - accuracy: 0.1290 - loss: 1.9462 - val_accuracy: 0.1464 - val_loss: 1.9477
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 140ms/step - accuracy: 0.1319 - loss: 1.9472 - val_accuracy: 0.1446 - val_loss: 1.9491
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 23s 162ms/step - accuracy: 0.1434 - loss: 1.9447 - val_accuracy: 0.1545 - val_loss: 1.9438
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 168ms/step - accuracy: 0.1438 - loss: 1.9435 - val_accuracy: 0.1848 - val_loss: 1.9314
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 171ms/step - accuracy: 0.1603 - loss: 1.9431 - val_accuracy: 0.1652 - val_loss: 1.9297
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 175ms/step - accuracy: 0.1846 - loss: 1.9285 - val_accuracy: 0.1571 - val_loss: 1.9309
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 179ms/step - accuracy: 0.1566 - loss: 1.9211 - val_accuracy: 0.1741 - val_loss: 1.9040
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 26s 183ms/step - accuracy: 0.1943 -

In [29]:
model_8 = Sequential()
model_8.add(Input(shape=(max_len, embedding_dim)))
model_8.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model_8.add(Dense(units=75, activation='relu'))
model_8.add(Dense(units=50, activation='relu'))
model_8.add(Dense(units=7, activation='softmax'))

model_8.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# Fit model_8
history_8 = model_8.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    callbacks=[EarlyStopper_4],
    verbose=1
)

Epoch 1/100
140/140 [==============================] - 107s 746ms/step - loss: 1.9459 - accuracy: 0.1373 - val_loss: 1.9456 - val_accuracy: 0.1500
Epoch 2/100
140/140 [==============================] - 98s 698ms/step - loss: 1.9453 - accuracy: 0.1406 - val_loss: 1.9456 - val_accuracy: 0.1393
Epoch 3/100
140/140 [==============================] - 103s 739ms/step - loss: 1.9424 - accuracy: 0.1440 - val_loss: 1.9465 - val_accuracy: 0.1527
Epoch 4/100
140/140 [==============================] - 104s 742ms/step - loss: 1.9419 - accuracy: 0.1473 - val_loss: 1.9458 - val_accuracy: 0.1500
Epoch 5/100
140/140 [==============================] - 106s 759ms/step - loss: 1.9375 - accuracy: 0.1589 - val_loss: 1.9317 - val_accuracy: 0.1500
Epoch 6/100
140/140 [==============================] - 102s 730ms/step - loss: 1.9389 - accuracy: 0.1587 - val_loss: 1.9447 - val_accuracy: 0.1473
Epoch 7/100
140/140 [==============================] - 101s 720ms/step - loss: 1.9354 - accuracy: 0.1658 - val_loss: 1.

In [79]:
EarlyStopper_5 = EarlyStopping(patience=15, restore_best_weights = True, verbose=1)

In [80]:
from keras.regularizers import l2

# Define the regularization parameter
l2_reg = 0.01

model_9 = Sequential()
model_9.add(Input(shape=(max_len, embedding_dim)))
model_9.add(Masking())
model_9.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_9.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(l2_reg)))
model_9.add(Dense(units=64, activation='relu', kernel_regularizer=l2(l2_reg)))
model_9.add(Dropout(0.4))
model_9.add(Dense(units=7, activation='softmax'))

model_9.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [81]:
# Fit model_9
history_9 = model_9.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    callbacks=[EarlyStopper_5],
    batch_size=32,
    shuffle=True,
    verbose=1
)

Epoch 1/100


2024-05-10 16:36:29.813519: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


140/140 ━━━━━━━━━━━━━━━━━━━━ 70s 413ms/step - accuracy: 0.1582 - loss: 4.0929 - val_accuracy: 0.1991 - val_loss: 2.1445
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 66s 469ms/step - accuracy: 0.1739 - loss: 2.1015 - val_accuracy: 0.1937 - val_loss: 1.9507
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 71s 505ms/step - accuracy: 0.1871 - loss: 1.9527 - val_accuracy: 0.2116 - val_loss: 1.9119
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 67s 476ms/step - accuracy: 0.1639 - loss: 1.9740 - val_accuracy: 0.1464 - val_loss: 1.9718
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 72s 515ms/step - accuracy: 0.1549 - loss: 1.9667 - val_accuracy: 0.1464 - val_loss: 1.9579
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 75s 539ms/step - accuracy: 0.1421 - loss: 1.9563 - val_accuracy: 0.1464 - val_loss: 1.9538
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 78s 559ms/step - accuracy: 0.1430 - loss: 1.9548 - val_accuracy: 0.1464 - val_loss: 1.9534
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 85s 610ms/step - accuracy: 0.1534 - loss: 1.952

In [82]:
model_10 = Sequential()
model_10.add(Input(shape=(max_len, embedding_dim)))
model_10.add(Masking())
model_10.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True, kernel_regularizer=l2(l2_reg)))
model_10.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model_10.add(Dense(units=64, activation='relu', kernel_regularizer=l2(l2_reg)))
model_10.add(Dropout(0.4))
model_10.add(Dense(units=7, activation='softmax'))

model_10.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [83]:
# Fit model_10
history_10 = model_10.fit(
    X_train_pad,
    y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    callbacks=[EarlyStopper_5],
    batch_size=32,
    shuffle=True,
    verbose=1
)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 55s 370ms/step - accuracy: 0.1517 - loss: 3.0352 - val_accuracy: 0.2071 - val_loss: 2.0867
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 54s 383ms/step - accuracy: 0.1745 - loss: 2.0500 - val_accuracy: 0.1732 - val_loss: 2.0655
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 59s 420ms/step - accuracy: 0.1670 - loss: 1.9967 - val_accuracy: 0.1848 - val_loss: 1.9543
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 58s 417ms/step - accuracy: 0.1787 - loss: 1.9608 - val_accuracy: 0.1723 - val_loss: 1.9710
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 58s 412ms/step - accuracy: 0.1598 - loss: 1.9587 - val_accuracy: 0.2036 - val_loss: 1.9297
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 58s 415ms/step - accuracy: 0.1788 - loss: 1.9480 - val_accuracy: 0.2000 - val_loss: 1.9293
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 64s 459ms/step - accuracy: 0.1759 - loss: 1.9443 - val_accuracy: 0.1384 - val_loss: 1.9754
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 69s 492ms/step - accuracy: 0.1671 -